In [ ]:
# !pip install xarray
# !pip install pyyaml
# !pip install scipy==1.12

# !pip install matplotlib
!pip install seaborn==0.11
# !pip install scikit-image==0.22
# !pip install stardist
# !pip install tensorflow


In [ ]:
import os

import sys, importlib
import pkg_resources

import logging
import warnings
import time
from copy import deepcopy
from typing import Tuple


import numpy as np
import xarray as xr
import pandas as pd

import math
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
# pkg_resources.require("seaborn==0.11")
import seaborn as sns

# pkg_resources.require("scikit-image==0.22")
import skimage.measure
import scipy

from stardist.models import StarDist2D, StarDist3D

from statannotations.Annotator import Annotator


import multiprocessing


from utils import circular_mean, circular_variance

from generic import load_image, save_image
importlib.reload(sys.modules['generic'])


logging.basicConfig(level = logging.INFO)
logging.root.setLevel(logging.INFO)
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)
log.info('Logger initialised')

import matplotlib.font_manager

from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:
plt.style.context('seaborn-paper')
# plt.style.use('seaborn-notebook')
# plt.style.use('seaborn-poster')
# plt.style.use('seaborn-talk')
set_matplotlib_formats('svg')
plt.rcParams.update(plt.rcParamsDefault)


linewidth = 0.25
titlesize = 'medium'
labelsize = 'small'
ticksize = 'x-small'

markersize = 10
scattersize = 5

palette_stages='gist_heat'

plt.rcParams.update({
    'figure.figsize': [5.9/2, 5.9/2 * 2/3],

    'text.usetex': False,
    'font.size': 10,
    'font.family': 'sans-serif',
    'font.sans-serif': 'Helvetica',

    'figure.titlesize': titlesize,
    'legend.title_fontsize': labelsize,
    'legend.fontsize': ticksize,
    'axes.labelsize': labelsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize,
    'ytick.labelsize': ticksize,

    'figure.autolayout': False,

    'axes.linewidth': linewidth,
    'xtick.major.width': linewidth,
    'xtick.minor.width': 0.8*linewidth,
    'ytick.major.width': linewidth,
    'ytick.minor.width': 0.8*linewidth,
    'grid.linewidth': linewidth,

    'patch.linewidth': linewidth,

    'lines.markersize': scattersize
})

    

from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:
from matplotlib import font_manager
matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype', fontext='ttf')
font_manager.findfont('Helvetica', fallback_to_default=False)

In [ ]:
def interpolate_z(im: xr.DataArray, res_z=None, **kwargs):
    im -= im.min(dim=['x', 'y'])
    im /= im.max(dim=['x', 'y'])
    im = im.where(~np.isnan(im), 0.)

    if res_z is None:
        res_z = im.attrs.get(f"resolution_x")
    N = int((im.z[-1] + im.z[0])*res_z)
    zs = np.linspace(0, N, N+1) / res_z

    __kwargs = dict(**kwargs)
    __kwargs['method'] = __kwargs.get('method', 'quadratic')
    __kwargs['assume_sorted'] = __kwargs.get('assume_sorted', True)
    im = im.interp(z=zs, **__kwargs)
    im = im.dropna(dim='z', how='all')

    im.attrs['resolution_z'] = np.float32(1./(im.z[1] - im.z[0]).data)
    im.attrs['spacing'] = np.float32((im.z[1].data - im.z[0].data))
    return im

In [ ]:
def flood_mask(mask):
    """Applies flood_fill in every z-slice from every corner with value False
    """

    flooded_mask = []
    for z in mask.z:
        tmp = mask.sel(z=z).data
        tmp = np.where(~np.isnan(tmp), tmp, 0.)

        tmp = skimage.segmentation.flood_fill(tmp, (0, 0), -1)
        tmp = skimage.segmentation.flood_fill(tmp, (0, -1), -1)
        tmp = skimage.segmentation.flood_fill(tmp, (-1, 0), -1)
        tmp = skimage.segmentation.flood_fill(tmp, (-1, -1), -1)

        tmp = (tmp + 1) > 0        
        flooded_mask.append(xr.zeros_like(mask.sel(z=z), dtype=bool) + tmp)
    flooded_mask = xr.concat(flooded_mask, dim='z').transpose('x', 'y', 'z')
    flooded_mask.attrs = mask.attrs
    return flooded_mask

In [ ]:
def volume(regionmask, *, intensity=None, **kwargs):
    for k, v in kwargs:
        print(k, v)
    if intensity is not None:
        return intensity[regionmask].sum()
    else:
        return regionmask.sum()

In [ ]:
config = dict({
    'paths': {
        'basepath': '/home/julian/image_analysis/data/Volume_v3_manuals',
        'originals': '1_tifs',
        'segmentations': '2_manuals',
        'overwrite_exisiting': False,
        'skip_new': False,
        'split': ['stage', 'sample', 'PD', 'cut', 'frame', 'id'],
        'data_out': '81_data',
        'analysis_out': '91_analysis',
    },
    'image_channels': {
        'HC': 0,
        'actin': 1,
        'nuclei': 2,
    }
})

In [ ]:
# load images and segmentations
data = pd.DataFrame()

basepath = config['paths'].get('basepath')
segmentations = config['paths'].get('segmentations')
data_out = config['paths'].get('data_out')
analysis = config['paths'].get('analysis_out')
filenames = []

overwrite = config['paths'].get('overwrite_exisiting')

for filename in os.listdir(os.path.join(basepath, segmentations)):
    f = os.path.join(basepath, segmentations, filename)
    # checking if it is a file
    if os.path.isfile(f) and filename[-4:] == '.tif':
        filenames.append(filename)

# filenames = filenames[0:3]
# filenames = [
#     'E15_d_25p_4_0000_1.tif'
# ]

print(filenames)

HC_channel = config['image_channels'].get('HC')
actin_channel = config['image_channels'].get('actin')
nuclei_channel = config['image_channels'].get('nuclei')

properties = [
    'area', 
    'area_bbox',
    'area_convex',
    'area_filled',
    'axis_major_length',
    'axis_minor_length',
    'centroid',
    'eccentricity',
    'extent',
    'label',
    'orientation',
    'perimeter',
]

for i, filename in enumerate(filenames):
    log.info(f"Progress: {i+1}/{len(filenames)}")
    data_path = os.path.join(
        basepath,
        data_out,
        f"{filename.split('.')[0]}.csv"
    )
    
    if not overwrite and os.path.isfile(data_path):
        log.info("Found data for image {}. Restoring data ...".format(filename))
        
        _data = pd.read_csv(data_path)
        data = pd.concat([data, _data])

    elif not config['paths'].get('skip_new'):
        log.info("Loading image {} ...".format(filename))
        im = load_image(
            base_dir = basepath,
            in_dir=segmentations,
            filename=filename,
            dims=['z', 'c', 'y', 'x'],
            normalise=True,
            logger=log
        ).squeeze().sel(c=[HC_channel, actin_channel, nuclei_channel])
    
        im = im.assign_coords(c=['HC', 'actin', 'nuclei'])
    
        mask = xr.where(im.sel(c='HC') > 0.5, 0., 1, keep_attrs=True)
        mask = flood_mask(mask)
        im = xr.concat([mask, im.sel(c='actin'), im.sel(c='nuclei')], dim='c', combine_attrs='no_conflicts')
    
        im = im.transpose('x', 'y', 'z', 'c')
        im -= im.min(dim=['x', 'y'])
        im /= im.max(dim=['x', 'y'])
    
        # im = interpolate_z(im, method='linear')
    
        HC_labels, _ = scipy.ndimage.label(im.sel(c='HC') > 0.5)
        HC_labels = xr.zeros_like(im.sel(c='HC')) + HC_labels
        HC_labels.attrs = im.attrs
    
        regionprops = skimage.measure.regionprops(
            HC_labels.data.astype(int),
            spacing=(
                im.attrs['resolution_x']**-1,
                im.attrs['resolution_y']**-1,
                im.attrs['resolution_z']**-1,
            ),
            cache=False
        )
    
        _data = pd.DataFrame()
        for region in regionprops:
            label = region['label']
            _data.loc[label, 'volume'] = region['area']
            _data.loc[label, 'volume_bbox'] = region['area_bbox']
            _data.loc[label, 'volume_convex'] = region['area_convex']
            _data.loc[label, 'volume_filled'] = region['area_filled']
            _data.loc[label, 'axis_major_length'] = region['axis_major_length']
            _data.loc[label, 'axis_minor_length'] = region['axis_minor_length']
            centroid = region['centroid']
            _data.loc[label, 'centroid_x'] = centroid[0]
            _data.loc[label, 'centroid_y'] = centroid[1]
            _data.loc[label, 'centroid_z'] = centroid[2]
            # _data.loc[label, 'eccentricity'] = region['eccentricity']
            _data.loc[label, 'extent'] = region['extent']
            _data.loc[label, 'feret_diameter_max'] = region['feret_diameter_max']
            _data.loc[label, 'cell_id'] = region['label']
            # _data.loc[label, 'orientation'] = region['orientation']
            # _data.loc[label, 'perimeter'] = region['perimeter']
    
    
        im = xr.concat([HC_labels, im.sel(c='actin'), im.sel(c='nuclei')], dim='c', combine_attrs='no_conflicts')
        save_image(
            im,
            normalise=True,
            base_dir=basepath,
            out_dir="3_recomposition",
            filename=filename
        )
    
    
        HC_labels = HC_labels.where(HC_labels > 0)
    
    
        iter = (config['paths'].get('split'), filename.split('.')[0].split('_'))
    
        _data['filename'] = filename
        _data['file_id'] = i
        for k, v in zip(iter[0], iter[1]):
            _data[k]=v

        _data.to_csv(data_path)
    
        data = pd.concat([data, _data])
    
        it = 2
        N = int(np.floor(len(HC_labels.z) / it))
        fig, axes = plt.subplots(N+1, 1, figsize=[6, N * 2])
        for z, ax in enumerate(axes[:-1]):
            ax.set_aspect(1)
            ax.imshow(HC_labels.isel(z=z*it).transpose('y', 'x') % 10, interpolation='None', cmap='tab10', vmin=0, vmax=9)
        axes[-1].imshow(HC_labels.isel(z=-1).transpose('y', 'x') % 10, interpolation='None', cmap='tab10', vmin=0, vmax=9)
    
    
        plt.savefig(os.path.join(basepath, "3_recomposition", f"{filename[:-4]}.png"))
        plt.close()
    else:
        log.warn("Skipping image {} ...".format(filename))
        

data['radius_cylinder'] = (data['volume'] / math.pi / data['axis_major_length'])**0.5
data['area_cylinder'] = data['radius_cylinder']**2 * math.pi
data['area_minor_axis'] = (data['axis_minor_length'] / 2)**2 * math.pi

data['stage_'] = data['stage'].apply(lambda s: int(s[1:]))
data['PD_'] = data['PD'].apply(lambda s: int(s[:-1]))
data.sort_values(by=['stage_', 'PD_', 'sample', 'cut', 'frame', 'file_id', 'cell_id'], inplace=True)

def grp_stages(stage: str):
    if stage == 'E8' or stage == 'E9':
        return 'E8-9'
    if stage == 'E10' or stage == 'E11':
        return 'E10-11'
    if stage == 'E12' or stage == 'E13':
        return 'E12-13'
    if stage == 'E14' or stage == 'E15':
        return 'E14-15'
    return stage
data['stage_grp'] = data['stage'].apply(grp_stages)

data['global_cell_id'] = np.asarray(data['cell_id'].max() * data['file_id'] + data['cell_id'], dtype=int)
# print(data.to_string())

data.to_csv(os.path.join(basepath, data_out, "summary.csv"))


grp = ['file_id', 'filename', 'sample', 'stage', 'stage_grp', 'stage_', 'PD', 'PD_', 'cut', 'frame']
data_grp = data.groupby(by=grp)
tissue_data = data_grp.mean()

for col in tissue_data.columns:
    tissue_data[f"{col}__std"] = data_grp[col].std()
tissue_data = tissue_data.reset_index()

tissue_data.sort_values(by=['stage_', 'PD_', 'sample', 'cut', 'frame', 'file_id', 'cell_id'], inplace=True)
tissue_data.to_csv(os.path.join(basepath, data_out, "tissue_summary.csv"))

print(tissue_data.loc[tissue_data['PD']=='75p'])

im = None
del im
mask = None
del mask
HC_labels = None
del HC_labels
regionprops = None
del regionprops
_data = None
del _data

In [ ]:
data['PD'].unique()

In [ ]:
plots=np.asarray([
    ['volume', f'HC volume [$\mu m^3$]', 'volume'],
    ['axis_major_length', f'HC long axis [$\mu m$]', 'height'],
    ['radius_cylinder', f'HC simulated radius [$\mu m$]', 'radius_cylinder'],
    ['area_cylinder', f'HC crosssection \n estim. as cylinder [$\mu m^2$]', 'area_cylinder'],
    ['axis_minor_length', f'HC short axis [$\mu m$]', 'width'],
    ['area_minor_axis', f'HC crosssection area (minor axis) [$\mu m^2$]', 'area_crosssection'],
])

figsizes=dict({
    'volume': [2.2, 1.6],
    'height': [1.7, 1.7],
    'width': [1.7, 1.7],
    'area_cylinder': [1.7, 1.7],
    'area_crosssection': [1.7, 1.7],
})

for x, suffix in zip (['stage_grp', 'stage'], ['', '__']):
    box_pairs = []
    stages = tissue_data[x].unique()
    for i in range(len(stages)-1):
        box_pairs.append((stages[i], stages[i+1]))
    
    for y, ylabel, title in zip(plots[:,0], plots[:,1], plots[:,2]):
        fig, ax = plt.subplots(1, 1, figsize=figsizes.get(title, [3, 2]))
        sns.swarmplot(
            data=tissue_data,
            x=x,
            hue='PD',
            hue_order=['0p', '10p', '25p', '50p', '75p'],
            palette='Reds',
            dodge=True,
            y=y,
        )
        # for stage, path in zip(tissue_data[x].unique(), ax.collections):
        #     points = path.get_offsets()
        #     print(f"hue: {path.get_label()}; y-values: {stage, points[:, 0], points[:, 1]}")
        #     ys = tissue_data.loc[tissue_data[x]==stage, [y, f"{y}__std"]]
        #     ys = ys.sort_values(by=y)
        #     if not np.allclose(np.asarray(ys[y]), points[:, 1]):
        #         print(np.asarray(ys[y]))
        #         print(points[:, 1])
        #         raise RuntimeError("No match found")
        #     ax.errorbar(
        #         x=points[:,0],
        #         y=points[:,1],
        #         yerr=ys[f"{y}__std"],
        #         color='gray',
        #         ls='none',
        #         elinewidth=linewidth
        #     )

        annotator = Annotator(
            ax,
            box_pairs,
            data=data,
            x=x,
            y=y,
            # hue='PD'
        )
        annotator.configure(
            test='Mann-Whitney',
            line_width=0.75,
        )
        annotator.apply_and_annotate()
        
        sns.pointplot(
            data=tissue_data,
            x=x, 
            hue='PD',
            hue_order=['0p', '10p', '25p', '50p', '75p'],
            palette='Reds', 
            y=y,
            markers=['_', '_', '_', '_', '_'],
            join=False,
            dodge=True,
            ci='sd',
            errwidth=linewidth,
            scale=2
            # capsize=linewidth
            # kwargs=dict(
            #     marker='-'
            # )
        )
        ax.set_ylim(0, None)
        
        ax.set_xlabel('')
        
        ax.tick_params(axis='x', which='major', labelsize=labelsize)
        ax.set_ylabel(f'{ylabel}')
        
        plt.savefig(os.path.join(basepath, analysis, f"{title}{suffix}.svg"))


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[1.7, 1.7])

sns.scatterplot(
    data=tissue_data,
    x='area_cylinder', 
    y='axis_major_length',
    hue='stage_grp',
    # hue_order=['0p', '10p', '25p', '50p', '75p'],
    palette='Reds',
    style='PD',
    style_order=['0p', '10p', '25p', '50p', '75p']
)

ax.set_xlim([0., None])
ax.set_ylim([0., None])

ax.set_ylabel("Height [$\mu m$]")
ax.set_xlabel("Area [$\mu m^2$]")

plt.savefig(os.path.join(basepath, analysis, f"height_vs_area.svg"))

In [ ]:
raise

In [ ]:
tips = tissue_data.copy()
tips['day'] = tips['stage_grp']
tips['total_bill'] = tips['volume']
tips['sex'] = tips['PD']

tips= tips[['day', 'total_bill', 'sex']]

print(sns.__version__)
print(tips.head())

ax=sns.swarmplot(x="day", y="total_bill", hue="sex", data=tips,dodge=True)
sns.barplot(x="day", y="total_bill", hue="sex", data=tips,capsize=0.1,errwidth=1.25,alpha=0.25,ci=None)
xcentres=[0, 1,2]
delt=0.2
xneg=[x-delt for x in xcentres]
xpos=[x+delt for x in xcentres]
xvals=xneg+xpos
# xvals.sort()
yvals=np.asarray(tips.groupby(["day","sex"]).mean().total_bill)
yerr=np.asarray(tips.groupby(["day","sex"]).std().total_bill)


print(xvals)
print(yvals)
print(yerr)


ax.errorbar(
    x=xvals,
    y=yvals,
    yerr=yerr, 
    lw=0, 
    elinewidth=linewidth,
    color='gray'
) # ,capsize=4,errwidth=1.25,ecolor="red")
# (_, caps, _)=ax.errorbar(x=xvals,y=yvals,yerr=yerr,fmt=None,capsize=4,errwidth=1.25,ecolor="red")
# for cap in caps:
#     cap.set_markeredgewidth(2)

handles, labels = ax.get_legend_handles_labels()
l = plt.legend(handles[0:2], labels[0:2])
plt.ylabel("Total Bill ($)")
plt.xlabel("")
plt.show()

In [ ]:
Slices = dict({
    'E9_c_25p_1_0000_1.tif': {
        'x': [30, 130, 200],
        'y': [60, 100, 150],
        'z': [20, 50, 100]
    },
    'E11_c_25p_2_0002.tif': {
        'x': [530, 723],
        'y': [180],
        'z': [50, 100, 150]
    },
    'E15_b_25p_2_0000.tif': {
        'x': [733],
        'y': [375],
        'z': [15, 20],
    },
    'E10_b_25p_1_0001.tif': {
        'x': [587],
        'y': [487],
        'z': [20, 40],
    },
    'E15_b_25p_3_0001.tif': {
        'x': [547],
        'y': [243],
        'z': [64, 95],
    },
    'E9_d_25p_2_0000.tif': {
        'x': [527],
        'y': [238],
        'z': [80, 128],
    }
})
Dims = dict({
    'x': ['c', 'x', 'y', 'z'],
    'y': ['c', 'y', 'z', 'x'],
    'z': ['c', 'z', 'y', 'x'],
})

for filename in filenames:
    im = Images[filename]

    # save_image(
    #     im,
    #     normalise=True,
    #     base_dir=basepath,
    #     out_dir="1001_interpolate_z",
    #     filename=filename
    # )

    # if filename in Slices.keys():
    #     slices=Slices[filename]

    #     for k, vals in slices.items():
    #         for val in vals:
    #             save_image(
    #                 im.isel({k: val}),
    #                 normalise=True,
    #                 dims=Dims[k],
    #                 base_dir=basepath,
    #                 out_dir="1004_cell_pose_train_iteration_2",
    #                 filename=(filename[:-4] + f"__{k}_{val}.tif")
    #             )

im = None
del im

In [ ]:
for filename in filenames:
    __im = Images[filename]
    im = __im.isel(c=[0, 1, 2])

    z_sel = dict(z=0.5, method='nearest')

    fig, ax = plt.subplots(1, 1)
    ax.imshow(im.sel(**z_sel).transpose('y', 'x', 'c'), interpolation='none', vmin=0, vmax=1)


    nuclei = im.sel(c=0)
    actin = im.sel(c=2)

    # exclude labels outside the HC-mask
    mask = xr.zeros_like(actin) + scipy.ndimage.gaussian_filter(actin.data, [25, 25, 1])
    mask = mask > mask.mean(dim=['x', 'y'])

    boundary = np.where(~scipy.ndimage.binary_erosion(mask, structure=np.ones((20, 20, 1))), mask, 0)


    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(nuclei.where(mask, np.nan).sel(**z_sel).transpose('y', 'x'), interpolation='none')
    axes[1].imshow(actin.where(mask, np.nan).sel(**z_sel).transpose('y', 'x'), interpolation='none')

    blurred_nuclei = xr.zeros_like(nuclei) + scipy.ndimage.gaussian_filter(nuclei, 5)
    blurred_actin = xr.zeros_like(actin) + scipy.ndimage.gaussian_filter(actin, 5)


    expected_mean = 0.4
    nuclei_mask = phansalkar_slices(blurred_nuclei, window_size=25, q=4/expected_mean) 
    nuclei_mask = flood_mask(nuclei_mask)

    # nuclei_mask = xr.zeros_like(actin) + scipy.ndimage.binary_closing(nuclei_mask, structure=np.ones((5, 5, 1)))

    distances = scipy.ndimage.distance_transform_edt(nuclei_mask.data) # , sampling=[1, 1, 0]
    distances = skimage.filters.gaussian(distances, sigma=5)
    distances = xr.zeros_like(nuclei_mask, dtype=int) + distances

    # seed_distances = seed_distances.where(nuclei_mask)
    # coords = 

    # seed_coords = skimage.feature.peak_local_max(
    #     __im.isel(c=4).data,
    #     footprint=np.ones((5, 5, 5))
    # ) # , min_distance=5
    # seeds = np.zeros_like(distances, dtype=bool)
    
    seeds = __im.isel(c=4).data > 0.75
    seed_labels, _ = scipy.ndimage.label(seeds)
    seed_distances = scipy.ndimage.distance_transform_edt(~seeds) #, sampling=[5, 5, 1])
    seed_distances = xr.zeros_like(nuclei_mask, dtype=int) + seed_distances

    combined_distances = seed_distances - distances
    combined_distances -= combined_distances.min()
    
    labels = xr.zeros_like(nuclei_mask, dtype=int) + skimage.segmentation.watershed(-distances, seed_labels, mask=nuclei_mask.data)


    # exclude SC labels
    for id in np.unique(labels.where(__im.isel(c=4) - __im.isel(c=3))):
        if id == 0 or np.isnan(id):
            continue
        labels = labels.where(labels != id.data, 0)


    labels = labels.assign_coords(c=5)
    labels = labels.assign_coords(channel='nuclei_labels')
    labels.attrs = im.attrs

    if 'nuclei_labels' in __im.channel:
        __im[:, :, :, 5] = labels
    else:
        __im = xr.concat([__im, labels], dim='c')
        __im.attrs = Images[filename].attrs
    Images[filename] = __im


    save_image(
        __im.isel(c=[3, 1, 2]),
        normalise=True,
        base_dir=basepath,
        out_dir=nuclei_out,
        filename=filename
    )
    save_image(
        labels,
        normalise=True,
        base_dir=basepath,
        out_dir=nuclei_out,
        filename=f"{filename[:-4]}_mask.tif"
    )


    zs = im.z[::5]

    fig, axes = plt.subplots(4, zs.shape[0]+1, figsize=[20,6], sharex=True, sharey=True)
    for i, z in enumerate(zs):
        z_sel = dict(z=z, method='nearest')
        axes[0][i].imshow(blurred_nuclei.sel(**z_sel).transpose('y', 'x'), interpolation='None')
        axes[1][i].imshow(nuclei_mask.sel(**z_sel).transpose('y', 'x'), interpolation='None')
        axes[2][i].imshow(distances.sel(**z_sel).transpose('y', 'x'), interpolation='None')
        axes[3][i].imshow((labels.where(labels > 0) % 20).sel(**z_sel).transpose('y', 'x'), interpolation='None', cmap='tab20')

    z_sel = dict(z=im.z[-1], method='nearest')
    axes[0][-1].imshow(blurred_nuclei.sel(**z_sel).transpose('y', 'x'), interpolation='None')
    axes[1][-1].imshow(nuclei_mask.sel(**z_sel).transpose('y', 'x'), interpolation='None')
    axes[2][-1].imshow(distances.sel(**z_sel).transpose('y', 'x'), interpolation='None')
    axes[3][-1].imshow((labels.where(labels > 0) % 20).sel(**z_sel).transpose('y', 'x'), interpolation='None', cmap='tab20')
    

In [ ]:
for filename in filenames
    __im=Images[filename]
    im = __im.isel(c=[0, 1, 2])
    actin=im.isel(c=1)
    nuclei=im.isel(c=2)
    nuclei_labels=__im.isel(c=5)


    # --- Prepar images using blur -------------------------------------------------
    use_2_blur = True
    if use_2_blur:
        actin_mask = phansalkar_slices(
            xr.zeros_like(actin) + scipy.ndimage.gaussian_filter(actin, 25),
            window_size=15
        ) 
        actin = actin.where(actin_mask, 0.)
        nuclei = nuclei.where(actin_mask, 0.)

    blurred_nuclei = xr.zeros_like(nuclei) + scipy.ndimage.gaussian_filter(nuclei, 5)
    blurred_actin = xr.zeros_like(actin) + scipy.ndimage.gaussian_filter(actin, 5)


    # # --- detect nuclei using watershed on threshold image -------------------------
    # nuclei_mask = phansalkar_slices(blurred_nuclei, window_size=25)
    # nuclei_distances = scipy.ndimage.distance_transform_edt(nuclei_mask)
    # nuclei_distances = skimage.filters.gaussian(nuclei_distances, sigma=5)
    # # print(np.isnan(nuclei_distances).sum())
    # nuclei_distances = xr.zeros_like(nuclei) + nuclei_distances
    # # print(nuclei_distances.max())
    # # nuclei_distances = nuclei_distances.where(nuclei_distances > 3, 0.)
    # nuclei_coords = skimage.feature.peak_local_max(
    #     nuclei_distances.data,
    #     footprint=np.ones((5, 5, 5)),
    #     # min_distance=5,
    # ) 
    # print(nuclei_coords)
    # nuclei_seeds = np.zeros_like(nuclei_distances, dtype=bool)
    # nuclei_seeds[tuple(nuclei_coords.T)] = True
    # nuclei_markers, _ = scipy.ndimage.label(nuclei_seeds)

    # nuclei_labels = skimage.segmentation.watershed(
    #     -nuclei_distances,
    #     nuclei_markers,
    #     mask=nuclei_mask
    # )
    # nuclei_labels = xr.zeros_like(nuclei, dtype=int) + nuclei_labels
    # nuclei_labels = nuclei_labels.where(nuclei_labels > 0, np.nan)


    # --- detect HC actin from nuclei seeds ----------------------------------------

    seeds = nuclei_labels.copy()
    for mean in np.linspace(0.6, 0.4, 3):
        actin_mask = phansalkar_slices(blurred_actin, window_size=15, q=4/mean)

        # HC = actin and overlapping nuclei masks
        HC_mask = actin_mask.where(seeds == 0, 1)
        # for id in np.unique(seeds.where(actin_mask)):
        #     if id == 0 or np.isnan(id):
        #         continue
        #     HC_mask = HC_mask.where(seeds != id, 1)


        # actin mask distances
        actin_distances = scipy.ndimage.distance_transform_edt(actin_mask==0, sampling=[5, 10, 1])
        actin_distances = xr.zeros_like(actin) + actin_distances

        HC_distances = scipy.ndimage.distance_transform_edt(HC_mask==0, sampling=[5, 10, 1])
        HC_distances = xr.zeros_like(actin) + HC_distances


        # watershed from nuclei segmentation
        labels = xr.zeros_like(actin_mask, dtype=int) + skimage.segmentation.watershed(actin_distances.data, seeds.data, mask=HC_mask.data)
        labels.attrs = im.attrs

        seeds = labels.copy()

    save_image(
        labels,
        normalise=True,
        base_dir=basepath,
        out_dir=HC_out,
        filename=f"{filename[:-4]}_mask.tif"
    )

    labels = labels.assign_coords(c=6)
    labels = labels.assign_coords(channel='HC_labels')

    if 'HC_labels' in __im.channel:
        __im[:, :, :, 6] = labels
    else:
        __im = xr.concat([__im, labels], dim='c')
        __im.attrs = Images[filename].attrs
    Images[filename] = __im



    # --- create output -----------------------------------------------------------

    nuclei_labels = nuclei_labels.where(nuclei_labels > 0, np.nan)
    labels = labels.where(labels > 0, np.nan)
    actin_mask = actin_mask.where(actin_mask > 0, np.nan) - 1
    distances = distances.where(HC_mask, np.nan)

    # zs = im.z[::5]

    fig, axes = plt.subplots(4, zs.shape[0], figsize=[20,6], sharex=True, sharey=True)
    for i, z in enumerate(zs):
        z_sel = dict(z=z, method='nearest')
        axes[0][i].imshow(blurred_actin.sel(**z_sel).transpose('y', 'x'), interpolation='None')
        axes[1][i].imshow((nuclei_labels % 19 + 1).where(nuclei_labels > 0, actin_mask).sel(**z_sel).transpose('y', 'x'), interpolation='None', cmap='tab20')
        # axes[2][i].imshow(actin_mask.sel(**z_sel).transpose('y', 'x'), interpolation='None')
        axes[2][i].imshow(actin_distances.sel(**z_sel).transpose('y', 'x'), interpolation='None')
        axes[3][i].imshow((labels % 20).sel(**z_sel).transpose('y', 'x'), interpolation='None', cmap='tab20')



In [ ]:
raise

# HC segmentaion

In [ ]:
for filename in filenames:
    im = Images[filename]

    z_sel = dict(z=4.25, method='nearest')

    nuclei = im.sel(c=0)
    nuclei = nuclei.interp(z=np.linspace(0.05, 13.95, 140))

    actin = im.sel(c=2)
    actin = actin.interp(z=np.linspace(0.05, 13.95, 140))

    nuclei_mask = []
    for z in nuclei.z:
        nuclei_mask.append(xr.zeros_like(nuclei.sel(z=z)) + phansalkar(nuclei.sel(z=z).data))
    nuclei_mask = xr.concat(nuclei_mask, dim='z').transpose('x', 'y', 'z')

    flooded_nuclei = flood_mask(nuclei_mask)

    distances = scipy.ndimage.distance_transform_edt(flooded_nuclei.data)
    distances = xr.zeros_like(nuclei_mask, dtype=int) + distances

    # use manual seeds
    inverse_actin = np.ones_like(actin.data) - actin.data
    inverse_actin = skimage.filters.gaussian(inverse_actin, sigma=4)
    inverse_actin = xr.zeros_like(actin) + inverse_actin
    coords = skimage.feature.peak_local_max(inverse_actin.data)
    print(coords)
    max = np.asarray(coords, dtype=int)
    mask = np.zeros_like(distances, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = scipy.ndimage.label(mask)

    labels = xr.zeros_like(flooded_nuclei, dtype=int) + skimage.segmentation.watershed(-distances, markers, mask=flooded_nuclei.data)


    fig, axes = plt.subplots(2, 50, figsize=[50, 3], sharex=True, sharey=True)
    for i, z in enumerate(im.z):
        axes[0][i].imshow(im.sel(z=z, method='nearest', c=[1, 2, 0]).transpose('y', 'x', 'c'), interpolation='none')
        axes[1][i].imshow(inverse_actin.sel(z=z, method='nearest').transpose('y', 'x'), interpolation='none')
        
        tmp = distances.where(distances > 3).sel(z=z, method='nearest').transpose('y', 'x')
        axes[0][i].imshow(tmp.where(tmp > 0), interpolation='none')
        d = max[(max[:, 0] > i - 20) & (max[:, 0] < i + 20)]
        axes[0][i].scatter(d[:, 1], d[:, 2], color='red', s=1)
        d = max[max[:, 0] == i]
        axes[0][i].scatter(d[:, 1], d[:, 2], color='green', s=1)
        
        tmp = labels.sel(z=z, method='nearest').transpose('y', 'x')
        axes[1][i].imshow(tmp.where(tmp > 0) % 20, interpolation='none', cmap='tab20', vmin=0, vmax=19)
        d = max[(max[:, 0] > i - 20) & (max[:, 0] < i + 20)]
        axes[1][i].scatter(d[:, 1], d[:, 2], color='red', s=1)
        d = max[max[:, 0] == i]
        axes[1][i].scatter(d[:, 1], d[:, 2], color='green', s=1)


    distances = distances.assign_coords(c='distances')
    labels = labels.assign_coords(c='labels')
    result = xr.concat([im.interp(z=np.linspace(0.05, 13.95, 140)), distances, xr.where(labels > 0, labels % 20 + 1, 0, keep_attrs=True)], dim='c')

    save_image(
        result,
        normalise=False,
        base_dir=basepath,
        out_dir="105_nuclei",
        filename=filename
    )

In [ ]:
for filename in filenames:
    im = Images[filename]

    z_sel = dict(z=4.25, method='nearest')

    fig, ax = plt.subplots(1, 1)
    ax.imshow(im.sel(**z_sel, c=[1, 2, 0]).transpose('y', 'x', 'c'), interpolation='none')

    actin = im.sel(c=2)
    actin = actin.interp(z=np.linspace(0.05, 13.95, 140))
    HC_seeds = im.sel(c=3).interp(z=np.linspace(0.05, 13.95, 140))

    nuclei = im.sel(c=0).interp(z=np.linspace(0.05, 13.95, 140))
    nuclei_seeds = im.sel(c=1).interp(z=np.linspace(0.05, 13.95, 140))

    fig, axes = plt.subplots(1, 1)
    ax.imshow(actin.sel(**z_sel).transpose('y', 'x'), interpolation='none')

    actin_mask = []
    actin_blurred = xr.zeros_like(actin) + skimage.filters.gaussian(actin.data + 0.35 * nuclei.data, sigma=3)
    for z in actin.z:
        __actin_mask = xr.zeros_like(actin.sel(z=z)) + phansalkar(actin_blurred.sel(z=z).data)
        actin_mask.append(__actin_mask)
    nuclei_mask = xr.concat(actin_mask, dim='z')


    fig, axes = plt.subplots(1, 3, figsize=[15, 6])
    axes[0].imshow(im.sel(**z_sel, c=[1, 2, 0]).transpose('y', 'x', 'c'), interpolation='none')
    tmp = nuclei_mask.sel(**z_sel).transpose('y', 'x')
    axes[1].imshow(actin_blurred.sel(**z_sel).transpose('y', 'x'), interpolation='none')
    axes[2].imshow(tmp.where(tmp > 0), interpolation='none')

    # raise


    flooded_actin = []
    for z in actin.z:
        __tmp = actin_mask.sel(z=z).data

        __tmp = skimage.segmentation.flood_fill(__tmp, (0, 0), -1)
        __tmp = skimage.segmentation.flood_fill(__tmp, (0, -1), -1)
        __tmp = skimage.segmentation.flood_fill(__tmp, (-1, 0), -1)
        __tmp = skimage.segmentation.flood_fill(__tmp, (-1, -1), -1)

        __tmp = (__tmp + 1) > 0

        flooded_nuclei.append(xr.zeros_like(actin_mask.sel(z=z), dtype=bool) + __tmp)
    flooded_nuclei = xr.concat(flooded_nuclei, dim='z')
    # tmp = actin_mask.data # skimage.morphology.binary_closing(actin_mask.data)
    # tmp = skimage.segmentation.flood_fill(tmp, (0, 0, 0), -1) + 1
    # tmp = skimage.morphology.binary_closing(tmp > 0)
    distances = scipy.ndimage.distance_transform_edt(flooded_actin.data)
    distances = skimage.filters.gaussian(distances, sigma=5)
    distances = xr.zeros_like(actin_mask, dtype=int) + distances
    tmp = flooded_actin.where(distances > 3).astype(bool)
    coords = skimage.feature.peak_local_max(nuclei_seeds.data) # , min_distance=5
    max = np.asarray(coords.copy(), dtype=int)

    print(coords.shape)
    print(coords)

    # coords = scipy.signal.find_peaks()


    mask = np.zeros_like(distances, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = scipy.ndimage.label(mask)

    

    labels = xr.zeros_like(flooded_actin, dtype=int) + skimage.segmentation.watershed(-distances, markers, mask=flooded_actin.data)

    # print(max[max[:, 0] == 14][:, 1:3])
    # axes[1].imshow(im.isel(z=8, c=[1, 2, 0]).transpose('y', 'x', 'c'), interpolation='none')
    # tmp = __actin_mask.isel(z=8).transpose('y', 'x')
    # tmp = __actin_mask.isel(z=slice(8)).sum(dim='z').transpose('y', 'x')
    tmp = flooded_actin.where(distances > 3).sel(**z_sel).transpose('y', 'x')
    cbar = axes[1].imshow(tmp.where(tmp > 0), interpolation='none')
    tmp = distances.sel(**z_sel).transpose('y', 'x')
    cbar = axes[2].imshow(tmp.where(tmp > 0), interpolation='none')


    fig, axes = plt.subplots(2, 50, figsize=[50, 3], sharex=True, sharey=True)
    for i, z in enumerate(im.z):
        axes[0][i].imshow(im.sel(z=z, method='nearest', c=[1, 2, 0]).transpose('y', 'x', 'c'), interpolation='none')
        axes[1][i].imshow(im.sel(z=z, method='nearest', c=[1, 2, 0]).transpose('y', 'x', 'c'), interpolation='none')
        
        tmp = distances.where(distances > 3).sel(z=z, method='nearest').transpose('y', 'x')
        axes[0][i].imshow(tmp.where(tmp > 0), interpolation='none')
        d = max[(max[:, 0] > i - 20) & (max[:, 0] < i + 20)]
        axes[0][i].scatter(d[:, 1], d[:, 2], color='red', s=1)
        d = max[max[:, 0] == i]
        axes[0][i].scatter(d[:, 1], d[:, 2], color='green', s=1)
        
        tmp = labels.sel(z=z, method='nearest').transpose('y', 'x')
        axes[1][i].imshow(tmp.where(tmp > 0) % 20, interpolation='none', cmap='tab20', vmin=0, vmax=19)
        d = max[(max[:, 0] > i - 20) & (max[:, 0] < i + 20)]
        axes[1][i].scatter(d[:, 1], d[:, 2], color='red', s=1)
        d = max[max[:, 0] == i]
        axes[1][i].scatter(d[:, 1], d[:, 2], color='green', s=1)


    distances = distances.assign_coords(c='distances')
    labels = labels.assign_coords(c='labels')
    result = xr.concat([im.interp(z=np.linspace(0.05, 13.95, 140)), distances, xr.where(labels > 0, labels % 20 + 1, 0, keep_attrs=True)], dim='c')
    print(result)

    save_image(
        result,
        normalise=False,
        base_dir=basepath,
        out_dir="105_nuclei",
        filename=filename
    )


    # ax = plt.figure().add_subplot(projection='3d')  
    # print(nuclei_mask.shape)
    # x, y, z = np.ndarray.nonzero(nuclei_mask.data)
    # print(nuclei_mask.data)
    # # ax.voxels(nuclei_mask.data, x, y, z)
    

In [ ]:
raise

In [ ]:
def subtract_background(image, radius=50, light_bg=False):
    from skimage.morphology import white_tophat, black_tophat, disk
    str_el = disk(radius) #you can also use 'ball' here to get a slightly smoother result at the cost of increased computing time
    if light_bg:
        return xr.DataArray(black_tophat(image, str_el), coords=image.coords)
    else:
        return xr.DataArray(white_tophat(image, str_el), coords=image.coords)

In [ ]:
# # print(Images[filenames[0]])
# # print(Apices[filenames[0]])

# fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=[16, 4])

# image = Images[filenames[0]]

# ax1.imshow(
#     image.squeeze().transpose('y', 'x', 'c'),
#     interpolation='none',
#     extent=[image.x.min(), image.x.max(), image.y.max(), image.y.min()]
# )

# # nuclei = subtract_background(image.isel(c=0))
# # _nuclei = nuclei
# _actin=actin
# nuclei = xr.DataArray(skimage.filters.gaussian(_nuclei, sigma=2), coords=_nuclei.coords)
# actin = xr.DataArray(skimage.filters.gaussian(_actin, sigma=1), coords=_actin.coords)
# # print(nuclei)
# ax2.imshow(
#     nuclei.squeeze().transpose('y', 'x'),
#     interpolation='none',
#     extent=[nuclei.x.min(), nuclei.x.max(), nuclei.y.max(), nuclei.y.min()]
# )

# actin = image.isel(c=1)
# ax3.imshow(
#     actin.squeeze().transpose('y', 'x'),
#     interpolation='none',
#     extent=[actin.x.min(), actin.x.max(), actin.y.max(), actin.y.min()]
# )

# # print(actin.max().data)

# mask = nuclei > nuclei.quantile(0.6)


# # Now we want to separate the two objects in imagefigure_path = 'analysis/1_figures'
figure_path = os.path.abspath(os.path.join(basepath, figure_path))
# # mask = np.zeros(distance.shape, dtype=bool)
# # mask[tuple(coords.T)] = True
# markers, _ = scipy.ndimage.label(mask)
# labels = skimage.segmentation.watershed(-distance, markers, mask=mask)
# labels = xr.DataArray(labels, coords=_nuclei.coords)

# print(labels)

# # mask = skimage.morphology.binary_closing(mask)
# # mask = skimage.morphology.binary_dilation(mask)
# augmented = _nuclei.where(mask)#.where(actin < actin.quantile(0.75))
# # augmented = xr.DataArray(augmented, coords=_nuclei.coords)
# ax4.imshow(
#     labels.squeeze().transpose('y', 'x'),
#     interpolation='none',
#     extent=[augmented.x.min(), augmented.x.max(), augmented.y.max(), augmented.y.min()]
# )

In [ ]:
def coordinates_to_image(data, reference):
    image = xr.zeros_like(reference)
    Dx = (image.x[1] - image.x[0])
    Dy = (image.y[1] - image.y[0])

    for x, y in zip(image.x, data):
        if not np.isfinite(y) or not np.isfinite(x):
            continue
        image.loc[dict(
            x=image.x[int(np.floor(x/Dx))],
            y=image.y[int(np.floor(y/Dy))])
        ] = 1

    return image


def plot_result_to_file(boundaries):
    fig, (ax0, ax1) = plt.subplots(1, 2)
    fig.suptitle(boundaries.filename.data)

    filename = str(boundaries.filename.data)

    image = Images[filename].isel(c=2).squeeze()
    
    apex = boundaries['apex']
    base = boundaries['base']
    mid = boundaries['mid']
    HC_base = boundaries['HC_base']

    im_apex = coordinates_to_image(apex, image)
    im_base = coordinates_to_image(base, image)
    im_mid = coordinates_to_image(mid, image)
    im_HC_base = coordinates_to_image(HC_base, image)

    image = image.where(im_apex == 0.)
    image = image.where(im_base == 0.)
    image = image.where(im_HC_base == 0.)
    image = image.where(im_mid == 0.)

    im = ax0.imshow(
        image.squeeze().transpose('y', 'x'),
        interpolation='none',
        extent=[image.x.min(), image.x.max(), image.y.max(), image.y.min()]
    )
    im = ax1.imshow(
        image.squeeze().transpose('y', 'x'),
        interpolation='none',
        extent=[image.x.min(), image.x.max(), image.y.max(), image.y.min()]
    )

    mid = boundaries['mid'].dropna(dim='x')
    mid = mid.sel(x=slice(mid.x.min(), mid.x.max(), 20))
    slope = boundaries['slope'].sel(x=mid.x)
    height = boundaries['height_estimate'].sel(x=mid.x)
    ax0.set_title("h=%.2f" % (height.mean().data, ))
    ax0.quiver(
        mid.x,  
        mid,
        (height * slope), 
        - (height * np.sqrt(1 - slope**2)),
        height,
        cmap='Reds',
        angles='xy', scale_units='xy', scale=1, pivot='mid',
        headwidth=0, headlength=0, headaxislength=0,
    )

    HC_height = boundaries['HC_height_estimate'].sel(x=mid.x)
    ax1.set_title("h=%.2f" % (HC_height.mean().data, ))
    ax1.quiver(
        HC_base.sel(x=mid.x).x,  
        (HC_base + apex).sel(x=mid.x) / 2.,
        HC_height * slope, 
        - HC_height * np.sqrt(1 - slope**2),
        HC_height,
        cmap='Reds',
        angles='xy', scale_units='xy', scale=1, pivot='mid',
        headwidth=0, headlength=0, headaxislength=0,
    )
    
    ax0.set_xlim(0, 150)
    ax0.set_ylim(150, 0)
    ax1.set_xlim(0, 150)
    ax1.set_ylim(150, 0)

    fig.savefig(os.path.join(basepath, filename[:-4] + ".svg"))

In [ ]:
mean_window = 31

def import_data(filename):
    print("working on file {} : {} / {}. ".format(filename, filenames.index(filename)+1, len(filenames)))
    apex = Apices[filename]
    base = Bases[filename]
    HC_base = HC_bases[filename]
    
    apex_ys = []
    base_ys = []
    HC_base_ys = []

    for x in apex.x:
        column = apex.sel(x=x)
        column = column.where(column > 0, drop=True)
        if column.size > 0:
            apex_ys.append(column.y.mean().data)
        else:
            apex_ys.append(np.nan)

        column = base.sel(x=x)
        column = column.where(column > 0, drop=True)
        if column.size > 0:
            base_ys.append(column.y.mean().data)
        else:
            base_ys.append(np.nan)

        column = HC_base.sel(x=x)
        column = column.where(column > 0, drop=True)
        if column.size > 0:
            HC_base_ys.append(column.y.mean().data)
        else:
            HC_base_ys.append(np.nan)

    apex_ys = xr.DataArray(apex_ys, dims=['x'], coords=dict(x=apex.x))
    base_ys = xr.DataArray(base_ys, dims=['x'], coords=dict(x=apex.x))
    HC_base_ys = xr.DataArray(HC_base_ys, dims=['x'], coords=dict(x=apex.x))


    apex_smooth = apex_ys.rolling(center=True, x=mean_window).mean()
    base_smooth = base_ys.rolling(center=True, x=mean_window).mean()
    HC_base_smooth = HC_base_ys.rolling(center=True, x=mean_window).mean()

    mid__raw = (apex_ys + base_ys + HC_base_ys) / 3.
    mid = mid__raw.rolling(center=True, x=(mean_window-1)+1).mean()
    slope__raw = mid__raw.differentiate("x")
    slope = slope__raw.rolling(center=True, x=4*(mean_window-1)+1).mean()
    
    height = (base_smooth - apex_smooth) / (slope**2 + 1)**0.5
    HC_height = (HC_base_smooth - apex_smooth) / (slope**2 + 1)**0.5
    SC_height = height - HC_height

    result = xr.Dataset(
        {
            'apex__raw': apex_ys, 
            'apex': apex_smooth,
            'HC_base__raw': HC_base_ys, 
            'HC_base': HC_base_smooth,
            'base__raw': base_ys, 
            'base': base_smooth,
            'mid': mid, 
            'slope': slope,
            'height_estimate': height, 
            'HC_height_estimate': HC_height,
            'SC_height_estimate': SC_height
        },
        coords={
            'x': apex.x,
            'filename': filename
        },
        attrs={
            'smoothing_window': mean_window
        }
    )
    
    plot_result_to_file(result)

    return result

pool_obj = multiprocessing.Pool(processes=6)
data = xr.concat(pool_obj.map(import_data, filenames), dim='filename')
pool_obj.close()

data = data.assign(relative_HC_height=lambda data: data.HC_height_estimate / data.height_estimate)

split = ['stage', 'sample', 'position']
for i, s in enumerate(split):
    data = data.assign_coords(dict({
        s: ('filename', [f.split("_")[i] for f in data.filename.data])
    }))

data = data.assign_coords(dict(
    day=('filename', [int(s[1:]) for s in data.stage.data])
)).sortby('day')

print(data)

In [ ]:
tissue_data = data.mean('x').to_dataframe().reset_index()#.sort_values(by='day')

In [ ]:
tissue_data
data

In [ ]:
!pip install statannotations
from statannotations.Annotator import Annotator


In [ ]:
# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[6, 4])
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=[8,3])
fig.tight_layout()

sns.violinplot(
    data=tissue_data, 
    # data=data.to_dataframe().reset_index(),
    x='stage', order=['E9', 'E11', 'E15'],
    y='HC_height_estimate',
    # hue='position', 
    # hue_order=['25p'],
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax1
)
sns.scatterplot(
    data=tissue_data, #.loc[tissue_data['position'] == '25p'], 
    # data=data.to_dataframe().reset_index(),
    x='stage', #order=['E9', 'E11', 'E13', 'E15'],
    y='HC_height_estimate',
    color='black',
    s=5,
    # join=False, dodge=True,
    # errorbar='sd',
    # zorder=-np.inf,
    ax=ax1
)

sns.violinplot(
    data=tissue_data, 
    # data=data.to_dataframe().reset_index(),
    x='stage', order=['E9', 'E11', 'E15'],
    y='height_estimate',
    # hue='position', 
    # hue_order=['25p'],
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax2
)

sns.violinplot(
    data=tissue_data, 
    # data=data.to_dataframe().reset_index(),
    x='stage', order=['E9', 'E11', 'E15'],
    y='relative_HC_height',
    # hue='position', 
    # hue_order=['25p'],
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax3
)
sns.violinplot(
    data=tissue_data, 
    # data=data.to_dataframe().reset_index(),
    x='stage', order=['E9', 'E11', 'E15'],
    y='SC_height_estimate',
    # hue='position', 
    # hue_order=['25p'],
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax4
)
# sns.pointplot(
#     data=tissue_data, 
#     x='stage', order=['E9', 'E11', 'E13', 'E15'],
#     hue='position',
#     y='height_estimate',
#     hue_order=['25p'],
#     join=False, dodge=True,
#     errorbar='sd',
#     ax=ax2
# )
# sns.pointplot(
#     data=tissue_data, 
#     x='stage', order=['E9', 'E11', 'E13', 'E15'],
#     y='relative_HC_height',
#     hue='position',
#     hue_order=['25p'],
#     join=False, dodge=True,
#     errorbar='sd',
#     ax=ax3
# )

# ax1.set_ylim(0, 25)
# ax2.set_ylim(0, 60)
# ax3.set_ylim(0, 0.8)

ax1.legend().remove()

ax1.set_xticks(['E9', 'E11', 'E15'])

box_pairs=[
    ('E9', 'E11'),
    ('E11', 'E15'),
    # (('E9', '25p'), ('E11', '25p')),
    # (('E11', '25p'), ('E15', '25p')),
    # (('E11', '75p'), ('E15', '75p'))
]

annotator = Annotator(
    ax1,
    box_pairs,
    data=tissue_data, #.loc[tissue_data['position'] == '25p'],
    # data=data.to_dataframe().reset_index(),
    x='stage', #order=['E9', 'E11', 'E13', 'E15'],
    y='HC_height_estimate',
    # hue_order=['25p'],
    # hue='position',
    # split=False,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    # loc='outside',
)
annotator.apply_and_annotate()

annotator = Annotator(
    ax2,
    box_pairs,
    data=tissue_data, #.loc[tissue_data['position'] == '25p'],
    # data=data.to_dataframe().reset_index(),
    x='stage', #order=['E9', 'E11', 'E13', 'E15'],
    y='height_estimate',
    # hue_order=['25p'],
    # hue='position',
    # split=False,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    # loc='outside',
)
annotator.apply_and_annotate()

annotator = Annotator(
    ax3,
    box_pairs,
    data=tissue_data, #.loc[tissue_data['position'] == '25p'],
    # data=data.to_dataframe().reset_index(),
    x='stage', #order=['E9', 'E11', 'E13', 'E15'],
    y='relative_HC_height',
    # hue_order=['25p'],
    # hue='position',
    # split=False,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    # loc='outside',
)
annotator.apply_and_annotate()

annotator = Annotator(
    ax4,
    box_pairs,
    data=tissue_data, #.loc[tissue_data['position'] == '25p'],
    # data=data.to_dataframe().reset_index(),
    x='stage', #order=['E9', 'E11', 'E13', 'E15'],
    y='SC_height_estimate',
    # hue_order=['25p'],
    # hue='position',
    # split=False,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    # loc='outside',
)
annotator.apply_and_annotate()

ax1.set_xlabel("")
ax1.set_ylabel("HC height est. [$\mu$m]")

plt.savefig(f"../analysis/1_figures/lateral_summary.svg",format='svg')

In [ ]:
fig, ax1= plt.subplots(1, 1, figsize=[1.6,2])
fig.tight_layout()

sns.violinplot(
    data=tissue_data, 
    x='stage', 
    order=['E9', 'E11', 'E15'],
    y='HC_height_estimate',
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax1
)

# ax2.set_ylim(0, 60)
# ax3.set_ylim(0, 0.8)

ax1.legend().remove()

# ax1.set_xticks(['E9', 'E11', 'E15'])

box_pairs=[
    ('E9', 'E11'),
    ('E11', 'E15'),
    # (('E9', '25p'), ('E11', '25p')),
    # (('E11', '25p'), ('E15', '25p')),
    # (('E11', '75p'), ('E15', '75p'))
]

annotator = Annotator(
    ax1,
    box_pairs,
    data=tissue_data,
    x='stage',
    order=['E9', 'E11', 'E15'],
    y='HC_height_estimate',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
)
annotator.apply_and_annotate()

ax1.set_ylim(0, 35)
ax1.set_yticks(np.linspace(0, 30, 4))
ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(5))

ax1.set_xlabel("")
ax1.set_ylabel("HC height estimate [$\mu$m]")

plt.savefig(f"../analysis/1_figures/HC_height.svg",format='svg')

In [ ]:
fig, ax1= plt.subplots(1, 1, figsize=[1.6,2])
fig.tight_layout()

sns.violinplot(
    data=tissue_data, 
    x='stage', 
    order=['E9', 'E11', 'E15'],
    y='height_estimate',
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax1
)

# ax2.set_ylim(0, 60)
# ax3.set_ylim(0, 0.8)

ax1.legend().remove()

# ax1.set_xticks(['E9', 'E11', 'E15'])

box_pairs=[
    ('E9', 'E11'),
    ('E11', 'E15'),
    # (('E9', '25p'), ('E11', '25p')),
    # (('E11', '25p'), ('E15', '25p')),
    # (('E11', '75p'), ('E15', '75p'))
]

annotator = Annotator(
    ax1,
    box_pairs,
    data=tissue_data,
    x='stage',
    order=['E9', 'E11', 'E15'],
    y='height_estimate',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
)
annotator.apply_and_annotate()

ax1.set_ylim(0, 95)
ax1.set_yticks(np.linspace(0, 80, 5))
ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))

ax1.set_ylabel("Tissue height estimate [$\mu$m]")
ax1.set_xlabel("")

plt.savefig(f"../analysis/1_figures/tissue_height.svg",format='svg')

fig, ax1= plt.subplots(1, 1, figsize=[1.6,2])
fig.tight_layout()

mean_height = tissue_data.groupby(by='stage').apply(lambda data: data['HC_height_estimate'] / data['height_estimate'].mean()).reset_index()
print(mean_height)
sns.violinplot(
    data=mean_height, 
    x='stage', 
    order=['E9', 'E11', 'E15'],
    y='HC_height_estimate',
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax1
)

# ax2.set_ylim(0, 60)
# ax3.set_ylim(0, 0.8)

ax1.legend().remove()

# ax1.set_xticks(['E9', 'E11', 'E15'])

box_pairs=[
    ('E9', 'E11'),
    ('E11', 'E15'),
    # (('E9', '25p'), ('E11', '25p')),
    # (('E11', '25p'), ('E15', '25p')),
    # (('E11', '75p'), ('E15', '75p'))
]

annotator = Annotator(
    ax1,
    box_pairs,
    data=mean_height,
    x='stage',
    order=['E9', 'E11', 'E15'],
    y='HC_height_estimate',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
)
annotator.apply_and_annotate()

ax1.set_ylim(0, 1)
ax1.set_yticks(np.linspace(0, 1, 5))
# ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))



ax1.set_xlabel("")
ax1.set_ylabel("Relative HC height estimate [$\mu$m]")

plt.savefig(f"../analysis/1_figures/HC_rel_height__.svg",format='svg')

In [ ]:
fig, ax1= plt.subplots(1, 1, figsize=[1.6,2])
fig.tight_layout()

sns.violinplot(
    data=tissue_data, 
    x='stage', 
    order=['E9', 'E11', 'E15'],
    y='relative_HC_height',
    join=False, dodge=True,
    errorbar='sd',
    inner='stick',
    ax=ax1
)

# ax2.set_ylim(0, 60)
# ax3.set_ylim(0, 0.8)

ax1.legend().remove()

# ax1.set_xticks(['E9', 'E11', 'E15'])

box_pairs=[
    ('E9', 'E11'),
    ('E11', 'E15'),
    # (('E9', '25p'), ('E11', '25p')),
    # (('E11', '25p'), ('E15', '25p')),
    # (('E11', '75p'), ('E15', '75p'))
]

annotator = Annotator(
    ax1,
    box_pairs,
    data=tissue_data,
    x='stage',
    order=['E9', 'E11', 'E15'],
    y='relative_HC_height',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
)
annotator.apply_and_annotate()

ax1.set_ylim(0, 1)
ax1.set_yticks(np.linspace(0, 1, 5))
# ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))



ax1.set_xlabel("")
ax1.set_ylabel("Relative HC height estimate [$\mu$m]")

plt.savefig(f"../analysis/1_figures/HC_rel_height.svg",format='svg')

In [ ]:
# for filename in filenames:
#     fig, ax = plt.subplots(1, 1)
#     fig.suptitle(filename)
#     _im = Images[filename].isel(c=2).squeeze()
#     _im = _im.where(Apices[filename] == 0.)
#     _im = _im.where(Bases[filename] == 0.)
#     _im = _im.where(HC_bases[filename] == 0.)

#     im = ax.imshow(_im.squeeze().transpose('y', 'x'), interpolation='none')

In [ ]:
# pool_obj = multiprocessing.Pool(processes=6)
# pool_obj.map(plot_file, data.filename.data)
# pool_obj.close()

In [ ]:
raise

In [ ]:
# gaussian_blur = 10
# for filename in filenames:
#     seg = Apices[filename]
#     seg = scipy.ndimage.gaussian_filter(seg, gaussian_blur)
#     skeleton = skimage.morphology.skeletonize(xr.where(seg==0, 1, 0))
#     skeleton = xr.where(skeleton==1, 0, 1)
#     Apices[filename] = skeleton

#     seg = Bases[filename]
#     seg = scipy.ndimage.gaussian_filter(seg, gaussian_blur)
#     skeleton = skimage.morphology.skeletonize(xr.where(seg==0, 1, 0))
#     skeleton = xr.where(skeleton==1, 0, 1)
#     Bases[filename] = skeleton

#     seg = HC_bases[filename]
#     seg = scipy.ndimage.gaussian_filter(seg, gaussian_blur)
#     skeleton = skimage.morphology.skeletonize(xr.where(seg==0, 1, 0))
#     skeleton = xr.where(skeleton==1, 0, 1)
#     HC_bases[filename] = skeleton

In [ ]:
!pip install opencv-python
import cv2

In [ ]:


# Read the image
img = cv2.imread(os.path.join(basepath, '2a_apical', filename.split('.')[0], 'handCorrection.tif'), cv2.IMREAD_GRAYSCALE)
print(img)

print(f'dtype: {img.dtype}, shape: {img.shape}, min: {np.min(img)}, max: {np.max(img)}')

# Convert the image to grayscale
gray = img # cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # Apply edge detection
edges = cv2.Canny(gray, 50, 150)

# Plot the original image and the detected edges
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(img, cmap='gray')
ax1.set_title('Original Image')
ax2.imshow(edges, cmap='gray')
ax2.set_title('Detected Edges')

In [ ]:


# Read the image
img = cv2.imread(os.path.join(basepath, '2a_apical', filename), cv2.IMREAD_GRAYSCALE)
print(img)

print(f'dtype: {img.dtype}, shape: {img.shape}, min: {np.min(img)}, max: {np.max(img)}')

# Convert the image to grayscale
gray = img # cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # Apply edge detection
edges = cv2.Canny(gray, 50, 150)

# Plot the original image and the detected edges
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(np.where(edges < 200, img, 0), cmap='gray')
ax1.set_title('Original Image')
ax2.imshow(edges, cmap='gray')
ax2.set_title('Detected Edges')